# **Part 1**


1. Implementation of **Sobel edge detection algorithm** from scratch. This part also includes comparioson between the method implemented from scratch and the one provided by OpenCV library.
2. Implementation of a method that uses output image from point 1 and **counts number of poles** in the provided 'fence.jpg' image.